In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 19 13:29:30 2022

@author: xdata
"""
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import math
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
import warnings
import pdb
warnings.filterwarnings('ignore')
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 禁止hash随机化


set_seed(2022)
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print('训练集的大小为： ', len(train))
print('测试集的大小为： ', len(test))


train = train.dropna()
train.reset_index(drop=True,inplace=True)

label = 'Main_steam_flow_rate'

# =============================================================================
# ['时间', '主蒸汽流量', 'CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '一次风调门', '一次风量',
#        '主蒸汽流量设定值', '二次风调门', '二次风量', '引风机转速', '推料器启停', '推料器手动指令',
#        '推料器自动投退信号', '推料器自动指令', '氧量设定值', '汽包水位', '炉排启停', '炉排实际运行指令',
#        '炉排手动指令', '炉排自动投退信号', '给水流量']
# =============================================================================


训练集的大小为：  257398
测试集的大小为：  1800


In [2]:
train.columns = ['Time', 'Main_steam_flow_rate', 'CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
       'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_start_stop', 'Pusher_manual_command',
       'Pusher_automatic_throwback_signal', 'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_start_stop', 'Grate_actual_operation_command',
       'Grate_manual_command', 'Grate_automatic_throw-out_signal', 'Feedwater_flow']

test.columns = ['Time', 'CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
       'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_start_stop', 'Pusher_manual_command',
       'Pusher_automatic_throwback_signal', 'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_start_stop', 'Grate_actual_operation_command',
       'Grate_manual_command', 'Grate_automatic_throw-out_signal', 'Feedwater_flow']


In [3]:
len(train.columns), len(test.columns)

(23, 22)

In [4]:
data = pd.concat([train, test])

In [5]:
data['gas'] = data.CO_content + data.HCL_content + data.SO2_content + data.NOx_content
for f in ['Feedwater_flow', 'Oxygen_setpoint', 'Primary_air_volume', 'Main_steam_flow_rate_setting', 'Ladle_water_level']:
    shift_f = []
    shift_d = []
    for i in range(200):
        colname = f+'_shift_{}'.format(i+1)
        data[colname] = data[f].shift(i+1)
        shift_f.append(colname)

        if i>10:
            shift_d.append(colname)
        colname = f+'_diff_{}'.format(i+1)
        data[colname] = data[f].diff(i+1)
# =============================================================================
#         if (i+1)%50==0:
#             data[f+'_fore_{}_steps_mean'.format(i)] = data[shift_f].mean(1)
#             data[f+'_fore_{}_steps_max'.format(i)] = data[shift_f].max(1)
#             data[f+'_fore_{}_steps_min'.format(i)] = data[shift_f].min(1)
#             data[f+'_fore_{}_steps_std'.format(i)] = data[shift_f].std(1)
# =============================================================================

    # data[f+'_diff'] = data[f].diff(1)
    data[f+'_fore_7_steps_mean'] = data[shift_f].mean(1)
    data[f+'_fore_7_steps_max'] = data[shift_f].max(1)
    data[f+'_fore_7_steps_min'] = data[shift_f].min(1)
    data[f+'_fore_7_steps_std'] = data[shift_f].std(1)
    data.drop(shift_d, axis=1, inplace=True)

In [6]:
data['Grate_start_stop'] = data['Grate_start_stop'].map(int)
data['Pusher_start_stop'] = data['Pusher_start_stop'].map(int)
data['Pusher_automatic_throwback_signal'] = data['Pusher_automatic_throwback_signal'].map(int)
data['Grate_automatic_throw-out_signal'] = data['Grate_automatic_throw-out_signal'].map(int)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259182 entries, 0 to 1799
Columns: 1099 entries, Time to Ladle_water_level_fore_7_steps_std
dtypes: float64(1094), int64(4), object(1)
memory usage: 2.1+ GB


In [7]:
test = data.tail(1800)
train = data.iloc[:-1800, :]
train.reset_index(drop=True,inplace=True)

valid = train.iloc[-1800:, :]
train = train.iloc[:-1800, :]

valid.reset_index(drop=True, inplace=True)



features = train.columns.drop(['Time',label]).tolist()


# dtrain = lgb.Dataset(train[features],label=train[label])
# dvalid = lgb.Dataset(valid[features],label=valid[label])

# watchlist = [dtrain,dvalid]
# model = lgb.train(params,train_set=dtrain,num_boost_round=10000,early_stopping_rounds=1000,valid_sets=watchlist,verbose_eval=200)
model = XGBRegressor(max_depth=6,learning_rate=0.05,n_estimators=100,randam_state=42)
model.fit(train[features],train[label],
          eval_set=[(valid[features], valid[label])],
          verbose=10, eval_metric='rmse')


# feature_importance = pd.DataFrame()
# feature_importance['fea_name'] = features
# feature_importance['fea_imp'] = model.feature_importance()
# feature_importance = feature_importance.sort_values('fea_imp',ascending = False)




sub = test[['Time']]
sub['lgb'+label] = model.predict(test[features])
sub.reset_index(inplace=True)
sub.columns=['ID','Time','Steam_flow']

sub.to_csv('result.csv',index=False)




[15:09:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "randam_state" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:56.76507
[10]	validation_0-rmse:33.92941
[20]	validation_0-rmse:20.22317
[30]	validation_0-rmse:12.03365
[40]	validation_0-rmse:7.25612
[50]	validation_0-rmse:4.44612
[60]	validation_0-rmse:3.03129
[70]	validation_0-rmse:2.38811
[80]	validation_0-rmse:2.21866


KeyboardInterrupt: 